### Problem 1
#### Question 2 : Hyperparameter search

In [1]:
# USEFUL IMPORTS
import mlp_cupy as mlp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import sklearn.model_selection
import time
import math
import cupy as cp
import gc

In [2]:
# LOAD THE DATA
data = pd.read_csv("mnist_train.csv")
data2 = pd.read_csv("mnist_test.csv")

In [3]:
# SPLIT THEM INTO TRAIN VALID TEST SETS
X = data.values[:,1:]/255
Y = data.values[:,0]
X_test = data2.values[:, 1:]/255
Y_test = data2.values[:, 0]
X_train, X_valid, Y_train, Y_valid = sklearn.model_selection.train_test_split( X, Y, test_size=0.1666666, random_state=42)

# convert to cupy array
X_train = cp.array(X_train)
X_valid = cp.array(X_valid)
X_test = cp.array(X_test)

Y_train = cp.array(Y_train)
Y_valid = cp.array(Y_valid)
Y_test = cp.array(Y_test)

In [4]:
print("Train set shape ", X_train.shape)
print("Valid set shape ", X_valid.shape)
print("Test  set shape ", X_test.shape)

Train set shape  (50000, 784)
Valid set shape  (10000, 784)
Test  set shape  (10000, 784)


#### Time test

In [66]:
start = time.time()
model = mlp.MLP_2L(784, 784, 480, 10, init="glorot")
model.fit(X_train, Y_train, 1, 1, 0.001, validation_data=(X_valid, Y_valid), epsilon=0.05)
end = time.time()
duration = end - start

days = int(duration/(24 * 60 * 60))
duration = duration % (24 * 60 * 60)
hours = int(duration/( 60 * 60))
duration = duration % (60 * 60)
minutes = int(duration/60)

duration2 = 128 * 4 * duration
days2 = int(duration2/(24 * 60 * 60))
duration2 = duration2 % (24 * 60 * 60)
hours2 = int(duration2/( 60 * 60))
duration2 = duration2 % (60 * 60)
minutes2 = int(duration2/60)
print("One training epoch with {:d} samples using SGD takes {:d}d {:2d}h {:2d}m".format(X_train.shape[0],days, hours, minutes ))
print("Then the whole fine tuning should take {:d}d {:2d}h {:2d}m".format(days2, hours2, minutes2 ))

Input dimension 784	Layer 1 dimension 784	Layer 2 dimension 480	Output dimension 10	 Initilization method glorot	Activation function identity
Total number of parameters : 997050
Train on 1000 samples	Evaluate on 10000
 samples
Epoch 1/1		Total training time 0.0s
	Samples 1000/1000	Epoch time 24.14s	Accuracy 0.848	Loss 0.714	Valid accuracy 0.803	 Valid loss 0.794

Total training time 24.14s
One training epoch with 50000 samples using SGD takes 0d  0h  0m
Then the whole fine tuning should take 0d  8h 35m


##### Fine tuning the hyperparameters

In [79]:
train_start = time.time()
input_size = 784
output_size = 10
count = 0
models_perf = []
# print("{:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s}".format("hidden 1", "hidden 2", "activation" , "l_rate", "l1", "l2",  "accuracy"))
for n1 in [ 500, 784]:
    for n2 in [300, 480]:
        n_parameter = n1 * (input_size + 1) + n2 * (n1 + 1) + output_size * (n2 + 1)
        if n_parameter > 1e6 or n_parameter < 5e5: continue
        for activation in ["identity", "relu", "sigmoid", "tanh"]:
            for learning_rate in [0.0001, 0.001 ]:
                for l1 in [0, 1e-5]:
                    for l2 in [0, 1e-5]:
                        count +=1
                        print("Model {:d}/128".format(count))
                        model = mlp.MLP_2L(input_size, n1, n2, output_size, init="glorot", verbose=False)
                        
                        model.fit(X_train, Y_train, 2, 10, learning_rate, validation_data=(X_valid, Y_valid), epsilon=0.001 )
                        model.fit(X_train, Y_train, 3, 1, learning_rate, validation_data=(X_valid, Y_valid), epsilon=0.001 )
                        
                        model.save("models/model"+str(count))
                        perf = dict()
                        perf["n1"] = n1
                        perf["n2"] = n2
                        perf["activation"] = activation
                        perf["learning_rate"] = learning_rate
                        perf["l1"] = l1
                        perf["l2"] = l2
                        perf["accuracy"] = model.evaluate(X_valid, Y_valid)
                        
                        models_perf.append(perf)
                        del model
                        gc.collect()
train_end = time.time()
duration3 = train_end - train_start
days3 = int(duration3/(24 * 60 * 60))
duration3 = duration3 % (24 * 60 * 60)
hours3 = int(duration3/( 60 * 60))
duration3 = duration3 % (60 * 60)
minutes3 = int(duration3/60)

print("Then the whole fine tuning took {:d}d {:2d}h {:2d}m".format(days3, hours3, minutes3 ))

Model 1/128
Model 2/128
Model 3/128
Model 4/128
Model 5/128
Model 6/128
Model 7/128
Model 8/128
Model 9/128
Model 10/128
Model 11/128
Model 12/128
Model 13/128
Model 14/128
Model 15/128
Model 16/128
Model 17/128
Model 18/128
Model 19/128
Model 20/128
Model 21/128
Model 22/128
Model 23/128
Model 24/128
Model 25/128
Model 26/128
Model 27/128
Model 28/128
Model 29/128
Model 30/128
Model 31/128
Model 32/128
Model 33/128
Model 34/128
Model 35/128
Model 36/128
Model 37/128
Model 38/128
Model 39/128
Model 40/128
Model 41/128
Model 42/128
Model 43/128
Model 44/128
Model 45/128
Model 46/128
Model 47/128
Model 48/128
Model 49/128
Model 50/128
Model 51/128
Model 52/128
Model 53/128
Model 54/128
Model 55/128
Model 56/128
Model 57/128
Model 58/128
Model 59/128
Model 60/128
Model 61/128
Model 62/128
Model 63/128
Model 64/128
Model 65/128
Model 66/128
Model 67/128
Model 68/128
Model 69/128
Model 70/128
Model 71/128
Model 72/128
Model 73/128
Model 74/128
Model 75/128
Model 76/128
Model 77/128
Model 78

NameError: name 'duratio3' is not defined

#### Results

In [76]:
print("{:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s} | {:^10s}".format("hidden 1", "hidden 2", "activation" , "l_rate", "l1", "l2",  "accuracy"))
print("________________________________________________________________________________________________")
for perf in models_perf:
    print("{:^10d} | {:^10d} | {:^10s} | {:^10f} | {:^10f} | {:^10f} | {:^10f}".format(perf["n1"], perf["n2"] , perf["activation"], perf["learning_rate"], perf["l1"], perf["l2"], perf["accuracy"]))

 hidden 1  |  hidden 2  | activation |   l_rate   |     l1     |     l2     |  accuracy 
________________________________________________________________________________________________
   500     |    300     |  identity  |  0.000100  |  0.000000  |  0.000000  |  0.144400 
   500     |    300     |  identity  |  0.000100  |  0.000000  |  0.000010  |  0.059300 
   500     |    300     |  identity  |  0.000100  |  0.000010  |  0.000000  |  0.070700 
   500     |    300     |  identity  |  0.000100  |  0.000010  |  0.000010  |  0.101000 
   500     |    300     |  identity  |  0.001000  |  0.000000  |  0.000000  |  0.079600 
   500     |    300     |  identity  |  0.001000  |  0.000000  |  0.000010  |  0.180000 
   500     |    300     |  identity  |  0.001000  |  0.000010  |  0.000000  |  0.170400 
   500     |    300     |  identity  |  0.001000  |  0.000010  |  0.000010  |  0.099600 
   500     |    300     |    relu    |  0.000100  |  0.000000  |  0.000000  |  0.099300 
   500     | 